BUSINESS PROBLEM

A real estate agency want to cluster neighborhood in the city of Rome to give to their clients a tailored offer

DATA

We create a dataframe containing the addresses of the districts of rome. 
Then we use geocoder to get the latitude and longiude for them.


Neighborhood Candidates

Let's create latitude & longitude coordinates for centroids of our candidate neighborhoods. We will create a grid of cells covering our area of interest which is aprox. 12x12 killometers centered around Rome city center.

Let's first find the latitude & longitude of Rome city center, using specific, well known address and Geopy library

In [10]:
!pip install geopy


LIBRARIES

In [11]:
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')
print('Folium installed')
print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.
Folium installed
Libraries imported.


In [12]:
data = [['Flaminio', 'Rome', 'Italy'], ['Parioli', 'Rome', 'Italy'], ['Pinciano', 'Rome', 'Italy'], ['Salario', 'Rome', 'Italy'],
       ['Nomentano', 'Rome', 'Italy'], ['Tiburtino', 'Rome', 'Italy'], ['Prenestino-Labicano', 'Rome', 'Italy'], ["Tuscolano", 'Rome', 'Italy'],
       ['Appio-Latino', 'Rome', 'Italy'], ['Ostiense', 'Rome', 'Italy'], ["Portuense", 'Rome', 'Italy'], ['Gianicolense', 'Rome', 'Italy'],
       ['Aurelio', 'Rome', 'Italy'], ['Trionfale', 'Rome', 'Italy'], ['Della Vittoria', 'Rome', 'Italy'], ['Monte Sacro', 'Rome', 'Italy'],
       ['Prenestino-Centocelle', 'Rome', 'Italy'], ['Trieste', 'Rome', 'Italy'], ['Testaccio', 'Rome', 'Italy'], ['Ardeatino', 'Rome', 'Italy'],
       ['Collatino', 'Rome', 'Italy'], ['Pietralata', 'Rome', 'Italy'], ['Alessandrino', 'Rome', 'Italy'], ['Don Bosco', 'Rome', 'Italy'],
       ['Appio Claudio', 'Rome', 'Italy']]

df = pd.DataFrame(data, columns = ['Rione', 'Città', 'Stato'] )

df['ADDRESS'] = df[['Rione', 'Città', 'Stato']].apply(lambda x: ', '.join(x), axis=1)

df.head()



,Rione,Città,Stato,ADDRESS
0,Flaminio,Rome,Italy,"Flaminio, Rome, Italy"
1,Parioli,Rome,Italy,"Parioli, Rome, Italy"
2,Pinciano,Rome,Italy,"Pinciano, Rome, Italy"
3,Salario,Rome,Italy,"Salario, Rome, Italy"
4,Nomentano,Rome,Italy,"Nomentano, Rome, Italy"


In [17]:

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
df['location'] = df['ADDRESS'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)

In [18]:
df = df.drop(['location', 'point', 'altitude'], axis = 1)
df.head()

,Rione,Città,Stato,ADDRESS,latitude,longitude
0,Flaminio,Rome,Italy,"Flaminio, Rome, Italy",41.912829,12.477139
1,Parioli,Rome,Italy,"Parioli, Rome, Italy",41.925949,12.487320
2,Pinciano,Rome,Italy,"Pinciano, Rome, Italy",41.918781,12.485332
3,Salario,Rome,Italy,"Salario, Rome, Italy",41.922978,12.502636
4,Nomentano,Rome,Italy,"Nomentano, Rome, Italy",41.917002,12.512918


In [19]:
locator = Nominatim(user_agent= 'myGeocoder')
location = locator.geocode('Piazza Venezia, Rome, Italy')
rome_center = [location.latitude, location.longitude]
print(rome_center)

[41.8966277, 12.4821685]


In [20]:
print(location)

Piazza Venezia, Rione IX Pigna, Municipio Roma I, Roma, Roma Capitale, 00186, Italia


In [21]:
df.head(20)

,Rione,Città,Stato,ADDRESS,latitude,longitude
0,Flaminio,Rome,Italy,"Flaminio, Rome, Italy",41.912829,12.477139
1,Parioli,Rome,Italy,"Parioli, Rome, Italy",41.925949,12.487320
2,Pinciano,Rome,Italy,"Pinciano, Rome, Italy",41.918781,12.485332
3,Salario,Rome,Italy,"Salario, Rome, Italy",41.922978,12.502636
4,Nomentano,Rome,Italy,"Nomentano, Rome, Italy",41.917002,12.512918
5,Tiburtino,Rome,Italy,"Tiburtino, Rome, Italy",41.899855,12.523146
6,Prenestino-Labicano,Rome,Italy,"Prenestino-Labicano, Rome, Italy",41.885017,12.545122
7,Tuscolano,Rome,Italy,"Tuscolano, Rome, Italy",41.869865,12.539586
8,Appio-Latino,Rome,Italy,"Appio-Latino, Rome, Italy",41.867904,12.516418
9,Ostiense,Rome,Italy,"Ostiense, Rome, Italy",41.857283,12.477881


We have so obtained the latitude and longitude of the Rioni of Rome. Now we can plot the data that we have 

In [22]:
import folium

In [23]:
map1 = folium.Map(
    location=rome_center,
    tiles='cartodbpositron',
    zoom_start=12,
)
df.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]]).add_to(map1), axis=1)
map1

Foursquare

Now that we have our location candidates, let's use Foursquare API to get info on restaurants in each neighborhood.

We're interested in venues in 'food' category, but only those that are proper restaurants - coffe shops, pizza places, bakeries etc. are not direct competitors so we don't care about those. So we will include in out list only venues that have 'restaurant' in category name, and we'll make sure to detect and include all the subcategories of specific 'Italian restaurant' category, as we need info on Italian restaurants in the neighborhood.

Foursquare credentials are defined in hidden cell bellow.


In [24]:
CLIENT_ID = 'UESEKXB012XIMLVUGSJC51E03GBY5H5JGWKE21UKTVVQ4D0T' # your Foursquare ID
CLIENT_SECRET = 'MMH4GOUGNUQIZGAXY5GDNNFWTC5XYB0DKABE4TBVGXUMZOO1' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UESEKXB012XIMLVUGSJC51E03GBY5H5JGWKE21UKTVVQ4D0T
CLIENT_SECRET:MMH4GOUGNUQIZGAXY5GDNNFWTC5XYB0DKABE4TBVGXUMZOO1


In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


df.head()

,Rione,Città,Stato,ADDRESS,latitude,longitude
0,Flaminio,Rome,Italy,"Flaminio, Rome, Italy",41.912829,12.477139
1,Parioli,Rome,Italy,"Parioli, Rome, Italy",41.925949,12.487320
2,Pinciano,Rome,Italy,"Pinciano, Rome, Italy",41.918781,12.485332
3,Salario,Rome,Italy,"Salario, Rome, Italy",41.922978,12.502636
4,Nomentano,Rome,Italy,"Nomentano, Rome, Italy",41.917002,12.512918


In [27]:
venues = getNearbyVenues(names=df['ADDRESS'],
                               latitudes=df['latitude'],
                               longitudes=df['longitude']
                                  )

Flaminio, Rome, Italy
Parioli, Rome, Italy
Pinciano, Rome, Italy
Salario, Rome, Italy
Nomentano, Rome, Italy
Tiburtino, Rome, Italy
Prenestino-Labicano, Rome, Italy
Tuscolano, Rome, Italy
Appio-Latino, Rome, Italy
Ostiense, Rome, Italy
Portuense, Rome, Italy
Gianicolense, Rome, Italy
Aurelio, Rome, Italy
Trionfale, Rome, Italy
Della Vittoria, Rome, Italy
Monte Sacro, Rome, Italy
Prenestino-Centocelle, Rome, Italy
Trieste, Rome, Italy
Testaccio, Rome, Italy
Ardeatino, Rome, Italy
Collatino, Rome, Italy
Pietralata, Rome, Italy
Alessandrino, Rome, Italy
Don Bosco, Rome, Italy
Appio Claudio, Rome, Italy


In [28]:
print(venues.shape)
venues.head()

(482, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Flaminio, Rome, Italy",41.912829,12.477139,Obelisco Flaminio,41.910751,12.476385,Monument / Landmark
1,"Flaminio, Rome, Italy",41.912829,12.477139,Terrazza del Pincio,41.911204,12.479160,Plaza
2,"Flaminio, Rome, Italy",41.912829,12.477139,Piazza del Popolo,41.910683,12.476342,Plaza
3,"Flaminio, Rome, Italy",41.912829,12.477139,My Bags,41.912125,12.474830,Sandwich Place
4,"Flaminio, Rome, Italy",41.912829,12.477139,Hotel de Russie,41.910126,12.477775,Hotel


In [29]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Alessandrino, Rome, Italy",8,8,8,8,8,8
"Appio Claudio, Rome, Italy",27,27,27,27,27,27
"Appio-Latino, Rome, Italy",8,8,8,8,8,8
"Ardeatino, Rome, Italy",4,4,4,4,4,4
"Aurelio, Rome, Italy",10,10,10,10,10,10
"Collatino, Rome, Italy",4,4,4,4,4,4
"Della Vittoria, Rome, Italy",30,30,30,30,30,30
"Don Bosco, Rome, Italy",30,30,30,30,30,30
"Flaminio, Rome, Italy",30,30,30,30,30,30


In [30]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
manhattan_onehot = onehot[fixed_columns]

onehot.head()

,Accessories Store,Art Museum,Art Studio,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Bed & Breakfast,Beer Garden,Beer Store,Bistro,Boarding House,Boutique,Breakfast Spot,Brewery,Burger Joint,Bus Station,Café,Camera Store,Casino,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Comedy Club,Comfort Food Restaurant,Concert Hall,Cosmetics Shop,Cupcake Shop,Department Store,Dessert Shop,Diner,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Farmers Market,Fast Food Restaurant,Fountain,Fried Chicken Joint,Garden,Gastropub,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Lake,Light Rail Station,Lingerie Store,Lounge,Market,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Multiplex,Museum,Music Venue,Nightclub,Other Nightlife,Outdoor Sculpture,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pub,Record Shop,Restaurant,Road,Rock Club,Roman Restaurant,Russian Restaurant,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Soccer Field,Steakhouse,Supermarket,Sushi Restaurant,Temple,Tennis Court,Thai Restaurant,Theater,Toy / Game Store,Track,Train Station,Trattoria/Osteria,Turkish Restaurant,University,Wine Bar,Wine Shop,Zoo,Neighborhood
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Flaminio, Rome, Italy"
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Flaminio, Rome, Italy"
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Flaminio, Rome, Italy"
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Flaminio, Rome, Italy"
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Flaminio, Rome, Italy"


In [104]:
onehot.shape

(486, 117)

In [31]:
rome_grouped = onehot.groupby('Neighborhood').mean().reset_index()
rome_grouped

,Neighborhood,Accessories Store,Art Museum,Art Studio,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Bed & Breakfast,Beer Garden,Beer Store,Bistro,Boarding House,Boutique,Breakfast Spot,Brewery,Burger Joint,Bus Station,Café,Camera Store,Casino,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Comedy Club,Comfort Food Restaurant,Concert Hall,Cosmetics Shop,Cupcake Shop,Department Store,Dessert Shop,Diner,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Farmers Market,Fast Food Restaurant,Fountain,Fried Chicken Joint,Garden,Gastropub,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Lake,Light Rail Station,Lingerie Store,Lounge,Market,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Multiplex,Museum,Music Venue,Nightclub,Other Nightlife,Outdoor Sculpture,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pub,Record Shop,Restaurant,Road,Rock Club,Roman Restaurant,Russian Restaurant,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Soccer Field,Steakhouse,Supermarket,Sushi Restaurant,Temple,Tennis Court,Thai Restaurant,Theater,Toy / Game Store,Track,Train Station,Trattoria/Osteria,Turkish Restaurant,University,Wine Bar,Wine Shop,Zoo
0,"Alessandrino, Rome, Italy",0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.125,0.00,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.250000,0.000000,0.000000,0.000000,0.000,0.000000,0.00,0.125,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Appio Claudio, Rome, Italy",0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.037037,0.000000,0.000000,0.000000,0.000000,0.037037,0.000000,0.037037,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.074074,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.037037,0.000000,0.000000,0.037037,0.000000,0.000000,0.000000,0.000000,0.037037,0.000000,0.000000,0.000000,0.037037,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.074074,0.000000,0.000000,0.000000,0.000000,0.00,0.037037,0.037037,0.000000,0.000000,0.000000,0.000000,0.037037,0.000000,0.000000,0.00,0.000000,0.000000,0.037037,0.000000,0.00,0.000000,0.00,0.037037,0.000,0.00,0.000000,0.148148,0.000000,0.000000,0.000000,0.074074,0.000000,0.000000,0.000000,0.000000,0.037037,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.037037,0.000000,0.000000,0.000000,0.000,0.000000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Appio-Latino, Rome, Italy",0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.00,0.000000,0.000

In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [41]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = rome_grouped['Neighborhood']

for ind in np.arange(rome_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(rome_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(20)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Alessandrino, Rome, Italy",Supermarket,Pizza Place,Track,Electronics Store,Market,Restaurant,Performing Arts Venue,Dessert Shop,Department Store,Farmers Market
1,"Appio Claudio, Rome, Italy",Pizza Place,Café,Ice Cream Shop,Pub,Clothing Store,Supermarket,Lounge,Roman Restaurant,Miscellaneous Shop,Fast Food Restaurant
2,"Appio-Latino, Rome, Italy",Italian Restaurant,Pub,Supermarket,Thai Restaurant,Gym,Seafood Restaurant,Hotel,Department Store,Dessert Shop,Diner
3,"Ardeatino, Rome, Italy",Italian Restaurant,Café,Gym Pool,Plaza,Asian Restaurant,Garden,Dessert Shop,Diner,Dog Run,Donut Shop
4,"Aurelio, Rome, Italy",Italian Restaurant,Bar,Hostel,Gym,Grocery Store,Pool,Bus Station,Supermarket,Hotel,Toy / Game Store
5,"Collatino, Rome, Italy",Italian Restaurant,Supermarket,Gym,Plaza,Asian Restaurant,Fried Chicken Joint,Department Store,Dessert Shop,Diner,Dog Run
6,"Della Vittoria, Rome, Italy",Dessert Shop,Pizza Place,Plaza,Italian Restaurant,Ice Cream Shop,Clothing Store,Steakhouse,Japanese Restaurant,Gym / Fitness Center,Gastropub
7,"Don Bosco, Rome, Italy",Clothing Store,Pizza Place,Fast Food Restaurant,Plaza,Café,Supermarket,Italian Restaurant,Gym / Fitness Center,Playground,Chinese Restaurant
8,"Flaminio, Rome, Italy",Italian Restaurant,Hotel,Plaza,Pizza Place,Wine Shop,Museum,Monument / Landmark,Mediterranean Restaurant,Japanese Restaurant,Hotel Bar
9,"Gianicolense, Rome, Italy",Café,Pizza Place,Italian Restaurant,Mediterranean Restaurant,Plaza,Brewery,Ice Cream Shop,Garden,Ethiopian Restaurant,Donut Shop


Clusters

In [42]:
# set number of clusters
kclusters = 5

rome_grouped_clustering = rome_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(rome_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 3, 3, 0, 0, 2, 2, 3, 2])

In [43]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

rome_merged = df




In [44]:
rome_merged = rome_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on ='ADDRESS')

rome_merged.head(20) # check the last columns!

,Rione,Città,Stato,ADDRESS,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Flaminio,Rome,Italy,"Flaminio, Rome, Italy",41.912829,12.477139,3,Italian Restaurant,Hotel,Plaza,Pizza Place,Wine Shop,Museum,Monument / Landmark,Mediterranean Restaurant,Japanese Restaurant,Hotel Bar
1,Parioli,Rome,Italy,"Parioli, Rome, Italy",41.925949,12.487320,3,Italian Restaurant,Seafood Restaurant,Café,Sandwich Place,Hotel,Asian Restaurant,University,Ice Cream Shop,Gym / Fitness Center,Mobile Phone Shop
2,Pinciano,Rome,Italy,"Pinciano, Rome, Italy",41.918781,12.485332,3,Hotel,Italian Restaurant,Sushi Restaurant,Art Museum,Café,Dog Run,Lake,Museum,Outdoor Sculpture,Park
3,Salario,Rome,Italy,"Salario, Rome, Italy",41.922978,12.502636,3,Italian Restaurant,Seafood Restaurant,Cocktail Bar,Restaurant,Café,Hotel,Diner,Burger Joint,Bar,Nightclub
4,Nomentano,Rome,Italy,"Nomentano, Rome, Italy",41.917002,12.512918,3,Hotel,Italian Restaurant,Art Museum,Supermarket,Park,Diner,Science Museum,Pizza Place,Ice Cream Shop,Gym
5,Tiburtino,Rome,Italy,"Tiburtino, Rome, Italy",41.899855,12.523146,2,Italian Restaurant,Burger Joint,Gym / Fitness Center,Lounge,Other Nightlife,Paper / Office Supplies Store,Park,Concert Hall,Cocktail Bar,Plaza
6,Prenestino-Labicano,Rome,Italy,"Prenestino-Labicano, Rome, Italy",41.885017,12.545122,2,Pizza Place,Market,Dessert Shop,Grocery Store,Italian Restaurant,Wine Shop,Historic Site,Gym Pool,Department Store,Diner
7,Tuscolano,Rome,Italy,"Tuscolano, Rome, Italy",41.869865,12.539586,3,Pool,Italian Restaurant,Sandwich Place,Japanese Restaurant,Kebab Restaurant,Fried Chicken Joint,Fast Food Restaurant,Monument / Landmark,Park,Restaurant
8,Appio-Latino,Rome,Italy,"Appio-Latino, Rome, Italy",41.867904,12.516418,3,Italian Restaurant,Pub,Supermarket,Thai Restaurant,Gym,Seafood Restaurant,Hotel,Department Store,Dessert Shop,Diner
9,Ostiense,Rome,Italy,"Ostiense, Rome, Italy",41.857283,12.477881,2,Italian Restaurant,Café,Pizza Place,Clothing Store,Ice Cream Shop,Pub,Chinese Restaurant,Cocktail Bar,Russian Restaurant,Diner


In [37]:
# create map
map_clusters = folium.Map(location= rome_center, zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(rome_merged['latitude'], rome_merged['longitude'], rome_merged['Rione'], rome_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [49]:
df0 = df['Cluster Labels' == 0]

KeyError: False